In [8]:
import re
import random

lines = open('./data.txt', 'r').read().splitlines()
testlines = open('./testdata.txt', 'r').read().splitlines()

parser = re.compile(r'Blueprint (?P<bpnum>\d+): Each ore robot costs (?P<oreore>\d+) ore. Each clay robot costs (?P<clayore>\d+) ore. Each obsidian robot costs (?P<obsore>\d+) ore and (?P<obsclay>\d+) clay. Each geode robot costs (?P<geore>\d+) ore and (?P<geobs>\d+) obsidian.')

options = [
    "no_action",
    "ore",
    "clay",
    "obs",
    "geode",
]

def max_possible_geodes(state, time):
    base = time * state['geode_miners']
    add = ((time - 1) * time) / 2
    return base + add

class BP:
    def __init__(self, line:str) -> None:
        result = parser.match(line)
        self.num = int(result['bpnum'])
        self.ore_ore = int(result['oreore'])
        self.clay_ore = int(result['clayore'])
        self.obs_ore = int(result['obsore'])
        self.obs_clay = int(result['obsclay'])
        self.geo_ore = int(result['geore'])
        self.geo_obs = int(result['geobs'])

class P1_Factory:
    def __init__(self, bp:BP) -> None:
        self.bp = bp

        self.max_geodes = 0
    
    def available_options(self, state):
        output = []
        for option in options:
            if option == 'no_action':
                output.append(option)
            elif option == 'ore':
                if state['ore'] >= self.bp.ore_ore:
                    output.append(option)
            elif option == 'clay':
                if state['ore'] >= self.bp.clay_ore:
                    output.append(option)
            elif option == 'obs':
                if state['ore'] >= self.bp.obs_ore and state['clay'] >= self.bp.obs_clay:
                    output.append(option)
            elif option == 'geode':
                if state['ore'] >= self.bp.geo_ore and state['obs'] >= self.bp.geo_obs:
                    output.append(option)
        return output
    
    def run_sim(self, time, state, decision):
        if time == 0:
            self.max_geodes = max(self.max_geodes, state['geode'])
            return
        # elif max_possible_geodes(state, time) <= self.max_geodes:
        #     return
        
        if decision == 'ore':
            state['ore'] = state['ore'] - self.bp.ore_ore
        elif decision == 'clay':
            state['ore'] = state['ore'] - self.bp.clay_ore
        elif decision == 'obs':
            state['ore'] = state['ore'] - self.bp.obs_ore
            state['clay'] = state['clay'] - self.bp.obs_clay
        elif decision == 'geode':
            state['ore'] = state['ore'] - self.bp.geo_ore
            state['obs'] = state['obs'] - self.bp.geo_obs
        
        state['ore'] = state['ore'] + state['ore_miners']
        state['clay'] = state['clay'] + state['clay_miners']
        state['obs'] = state['obs'] + state['obs_miners']
        state['geode'] = state['geode'] + state['geode_miners']

        if decision == 'ore':
            state['ore_miners'] = state['ore_miners'] + 1
        elif decision == 'clay':
            state['clay_miners'] = state['clay_miners'] + 1
        elif decision == 'obs':
            state['obs_miners'] = state['obs_miners'] + 1
        elif decision == 'geode':
            state['geode_miners'] = state['geode_miners'] + 1

        options = self.available_options(state)
        random.shuffle(options)
        for o in options:
            self.run_sim(time - 1, state.copy(), o)

    def start_sim(self):
        start_time = 24
        start_state = {
            "ore": 0,
            "clay": 0,
            "obs": 0,
            "geode": 0,
            "ore_miners": 1,
            "clay_miners": 0,
            "obs_miners": 0,
            "geode_miners": 0,
        }
        options = self.available_options(start_state)
        for o in options:
            self.run_sim(start_time, start_state.copy(), o)
        
        return self.max_geodes

bps = [BP(line) for line in testlines]

factories = [P1_Factory(bp) for bp in bps]
geodes = [f.start_sim() for f in factories]

geodes

KeyboardInterrupt: 